# Ejercicio: Sesgo del modelo de datos desequilibrados
En este ejercicio, examinaremos más detenidamente los conjuntos de datos desequilibrados, qué efectos tienen sobre las predicciones y cómo pueden abordarse.

También emplearemos matrices de confusión para evaluar las actualizaciones de los modelos.

## Visualización de datos
Al igual que en el ejercicio anterior, utilizamos un conjunto de datos que representa diferentes clases de objetos encontrados en la montaña:

In [1]:
import pandas
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects_balanced.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

# Let's have a look at the data
dataset

,size,roughness,color,motion,label
0,50.959361,1.318226,green,0.054290,tree
1,60.008521,0.554291,brown,0.000000,tree
2,20.530772,1.097752,white,1.380464,tree
3,28.092138,0.966482,grey,0.650528,tree
4,48.344211,0.799093,grey,0.000000,tree
...,...,...,...,...,...
2195,1.918175,1.182234,white,0.000000,animal
2196,1.000694,1.332152,black,4.041097,animal
2197,2.331485,0.734561,brown,0.961486,animal
2198,1.786560,0.707935,black,0.000000,animal


Recuerde que tenemos un conjunto de datos desequilibrado. Algunas clases son mucho más frecuentes que otras:

In [2]:
import graphing # custom graphing code. See our GitHub repo for details

# Plot a histogram with counts for each label
graphing.multiple_histogram(dataset, label_x="label", label_group="label", title="Label distribution")

## Utilización de la clasificación binaria
Para este ejercicio construiremos un modelo de clasificación binaria. Queremos predecir si los objetos en la nieve son "excursionistas" o "no excursionistas".

Para ello, primero tenemos que añadir otra columna a nuestro conjunto de datos, y establecerla en Verdadero cuando la etiqueta original sea excursionista, y en Falso para cualquier otra cosa:

In [3]:
# Add a new label with true/false values to our dataset
dataset["is_hiker"] = dataset.label == "hiker"

# Plot frequency for new label
graphing.multiple_histogram(dataset, label_x="is_hiker", label_group="is_hiker", title="Distribution for new binary label 'is_hiker'")

Ahora tenemos solo dos clases de etiquetas en nuestro conjunto de datos, pero lo hemos hecho aún más desequilibrado. Entrenemos el modelo de bosque aleatorio usando is_hiker como la variable de destino, luego midamos su precisión en los conjuntos de entrenamiento y prueba:

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

# Custom function that measures accuracy on different models and datasets
# We will use this in different parts of the exercise
def assess_accuracy(model, dataset, label):
    """
    Asesses model accuracy on different sets
    """ 
    actual = dataset[label]        
    predictions = model.predict(dataset[features])
    acc = accuracy_score(actual, predictions)
    return acc

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=1, shuffle=True)

# define a random forest model
model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False)

# Define which features are to be used (leave color out for now)
features = ["size", "roughness", "motion"]

# Train the model using the binary label
model.fit(train[features], train.is_hiker)

print("Train accuracy:", assess_accuracy(model,train, "is_hiker"))
print("Test accuracy:", assess_accuracy(model,test, "is_hiker"))

Train accuracy: 0.9532467532467532
Test accuracy: 0.906060606060606


La precisión se ve bien tanto para el entrenamiento como para los conjuntos de prueba, pero recuerde que esta métrica no es una medida absoluta del éxito. Deberíamos trazar una matriz de confusión para ver cómo funciona realmente el modelo:

In [6]:
# sklearn has a very convenient utility to build confusion matrices
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Calculate the model's accuracy on the TEST set
actual = test.is_hiker
predictions = model.predict(test[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Create the list of unique labels in the test set, to use in our plot
# I.e., ['True', 'False',]
unique_targets = sorted(list(test["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending")
                    )

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

La matriz de confusión nos muestra que, a pesar de las métricas indicadas, el modelo no es increíblemente preciso.

De las 660 muestras presentes en el conjunto de pruebas (el 30% del total), predijo 29 falsos negativos y 33 falsos positivos.

Y lo que es más importante, fíjese en la fila inferior, que muestra lo que ocurrió cuando se mostró al modelo información sobre un excursionista: se equivocó de respuesta casi el 30% de las veces. Esto significa que no identificaría correctamente a casi el 30% de las personas de la montaña.

¿Qué ocurre si utilizamos este modelo para hacer predicciones sobre conjuntos equilibrados?

Carguemos un conjunto de datos con el mismo número de resultados para "excursionistas" y "no excursionistas", y utilicemos esos datos para hacer predicciones:

In [7]:
# Load and print umbiased set
#Import the data from the .csv file
balanced_dataset = pandas.read_csv('snow_objects_balanced.csv', delimiter="\t")

#Let's have a look at the data
graphing.multiple_histogram(balanced_dataset, label_x="label", label_group="label", title="Label distribution")

Este nuevo conjunto de datos está equilibrado entre las clases, pero para nuestros fines queremos que esté equilibrado entre excursionistas y no excursionistas.

Para simplificar, tomemos los excursionistas más una muestra aleatoria de los no excursionistas.

In [8]:
# Add a new label with true/false values to our dataset
balanced_dataset["is_hiker"] = balanced_dataset.label == "hiker"

hikers_dataset = balanced_dataset[balanced_dataset["is_hiker"] == 1] 
nonhikers_dataset = balanced_dataset[balanced_dataset["is_hiker"] == False] 
# take a random sampling of non-hikers the same size as the hikers subset
nonhikers_dataset = nonhikers_dataset.sample(n=len(hikers_dataset.index), random_state=1)
balanced_dataset = pandas.concat([hikers_dataset, nonhikers_dataset])

# Plot frequency for "is_hiker" labels
graphing.multiple_histogram(balanced_dataset, label_x="is_hiker", label_group="is_hiker", title="Label distribution in balanced dataset")

Como puede ver, la etiqueta is_hiker tiene el mismo número de Verdadero y Falso para ambas clases. Ahora estamos usando un conjunto de datos balanceado de clases. Ejecutemos predicciones en este conjunto usando el modelo previamente entrenado:

In [9]:
# Test the model using a balanced dataset
actual = balanced_dataset.is_hiker
predictions = model.predict(balanced_dataset[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Print accuracy using this set
print("Balanced set accuracy:", assess_accuracy(model,balanced_dataset, "is_hiker"))

Balanced set accuracy: 0.754


Como era de esperar, vemos una caída notable en la precisión al usar un conjunto diferente. Nuevamente, analicemos visualmente su rendimiento:

In [10]:
# plot new confusion matrix
# Create the list of unique labels in the test set to use in our plot
unique_targets = sorted(list(balanced_dataset["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending")
                    )

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

La matriz de confusión confirma la escasa precisión de este conjunto de datos, pero ¿por qué ocurre esto cuando teníamos métricas tan excelentes en los conjuntos de entrenamiento y prueba anteriores?

Recordemos que el primer modelo estaba muy desequilibrado. La clase "excursionista" representaba aproximadamente el 22% de los resultados.

Cuando se produce un desequilibrio de este tipo, los modelos de clasificación no tienen suficientes datos para aprender los patrones de la clase minoritaria y, como consecuencia, se inclinan hacia la clase mayoritaria.

Los conjuntos desequilibrados pueden abordarse de varias maneras:

Mejorando la selección de datos
Remuestreo del conjunto de datos
Utilizando clases ponderadas
En este ejercicio, nos centraremos en la última opción.

## Utilizar ponderaciones de clases para equilibrar el conjunto de datos
Podemos asignar pesos diferentes a las clases mayoritaria y minoritaria, según su distribución, y modificar nuestro algoritmo de entrenamiento para que tenga en cuenta esa información durante la fase de entrenamiento.

Así, penalizará los errores cuando la clase minoritaria esté mal clasificada, lo que en esencia "obligará" al modelo a aprender mejor sus características y patrones.

Para utilizar clases ponderadas, tenemos que volver a entrenar nuestro modelo utilizando el conjunto de entrenamiento original, pero esta vez diciéndole al algoritmo que utilice las ponderaciones a la hora de calcular los errores:

In [11]:
# Import function used in calculating weights
from sklearn.utils import class_weight

# Retrain model using class weights
# Using class_weight="balanced" tells the algorithm to automatically calculate weights for us
weighted_model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False, class_weight="balanced")
# Train the weighted_model using binary label
weighted_model.fit(train[features], train.is_hiker)

print("Train accuracy:", assess_accuracy(weighted_model,train, "is_hiker"))
print("Test accuracy:", assess_accuracy(weighted_model, test, "is_hiker"))

Train accuracy: 0.9525974025974026
Test accuracy: 0.9166666666666666



Después de usar las clases ponderadas, la precisión del tren se mantuvo casi igual, mientras que la precisión de la prueba mostró una pequeña mejora (aproximadamente el 1 %). Veamos si los resultados mejoran usando el conjunto balanceado para predicciones nuevamente:

In [12]:
print("Balanced set accuracy:", assess_accuracy(weighted_model, balanced_dataset, "is_hiker"))

# Test the weighted_model using a balanced dataset
actual = balanced_dataset.is_hiker
predictions = weighted_model.predict(balanced_dataset[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

Balanced set accuracy: 0.796


La precisión del conjunto equilibrado aumentó aproximadamente un 4%, pero aún así debemos intentar visualizar y comprender los nuevos resultados.

Matriz de confusión final
Ahora podemos trazar una matriz de confusión final, que representa las predicciones para un conjunto de datos equilibrado, utilizando un modelo entrenado en un conjunto de datos de clases ponderadas:

In [13]:
# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending")
                    )

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

Aunque los resultados puedan parecer un poco decepcionantes, ahora tenemos un 21% de predicciones erróneas (FNs + FPs), frente al 25% del experimento anterior.

Las predicciones correctas (TP + TN) han pasado del 74,7% al 78,7%.

¿Es significativa o no una mejora de alrededor del 4%?

Recordemos que disponíamos de relativamente pocos datos para entrenar el modelo, y que las características de que disponemos pueden seguir siendo tan similares para muestras diferentes (por ejemplo, los excursionistas y los animales tienden a ser pequeños, no rugosos y a moverse mucho), que a pesar de nuestros esfuerzos, el modelo sigue teniendo algunas dificultades para hacer predicciones correctas.

Sólo hemos tenido que cambiar una línea de código para obtener mejores resultados, ¡así que parece que el esfuerzo ha merecido la pena!

Resumen
Este ha sido un ejercicio largo, en el que hemos tratado los siguientes temas:

Creación de nuevos campos de etiqueta para poder realizar una clasificación binaria utilizando un conjunto de datos con múltiples clases.
Cómo el entrenamiento en conjuntos desequilibrados puede tener un efecto negativo en el rendimiento, especialmente cuando se utilizan datos no vistos de conjuntos de datos equilibrados.
Evaluación de los resultados de los modelos de clasificación binaria mediante una matriz de confusión.
Utilización de clases ponderadas para abordar los desequilibrios de clase al entrenar un modelo y evaluar los resultados.